In [ ]:
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu" -O tennis_court_det_dataset.zip

--2025-03-15 15:12:31--  https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A1708243355040
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.217.97
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.217.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7255696316 (6.8G) [application/octet-stream]
Saving to: ‘tennis_court_det_dataset.zip’

tennis_court_det_da  73%[=============>      ]   4.96G  --.-KB/s    in 21m 41s 

2025-03-15 15:34:14 (3.90 MB/s) - Read error at byte 5322571776/7255696316 (Operation timed out). Retrying.

--2025-03-15 15:34:15--  (try: 2)  https://drive.usercontent.google.com/download?id=1lhAaeQCmk2y440PmagA0KmIVBIysVMwu&export=download&authuser=0&confirm=t&uuid=3077628e-fc9b-4ef2-8cde-b291040afb30&at=APZUnTU9lSikCSe3NqbxV5MVad5T%3A17082

In [ ]:
!git clone https://github.com/yastrebksv/TennisCourtDetector.git

In [ ]:
%unzip tennis_court_det_dataset.zip

In [ ]:
# !cd TennisCourtDetector
%pip install -r requirements.txt

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import json
import cv2
import numpy as np

In [ ]:
devic = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h,w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        kps = np.array(items['kps']).flatten()
        kps = kps.astype(np.float32)

        # Might need to be adjusted later, for the pickleball size
        kps[::2] *= 224.0 / w # Adjust x coordinates
        kps[1::2] *= 224.0 / h # Adjust y coordinates

        return img, kps

In [ ]:
# Creating the model
model = models.resnet50(pretrained=True)
model.fc =  torch.nn.Linear(model.fc.in_features, 14*2) # Replaces the last layer might need to adjust the num of key points

In [ ]:
model = model.to(device)

In [ ]:
# Train Model
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs=20
for epoch in range(epochs):
    for i, (imgs,kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, iter {i}, loss: {loss.item()}")


In [ ]:
torch.save(model.stat_dict(), "keypoints_model.pth")